# **Elo Merchant Category Recommendation**



---



# Table of Contents


>*   [7. Advance Feature Engineering](#section7)
  *   [7.1. Dataset 1 - card_details_train](#section7.1)
  *   [7.2. Dataset 2 - card_details_test](#section7.2)
  *   [7.3. Dataset 4 - historical_transactions](#section7.3)
  *   [7.4. Dataset 5 - new_transactions](#section7.4)









---



# Importing Libraries

In [ ]:
# importing the required libraries
import numpy as np
import pandas as pd
from matplotlib.dates import DateFormatter
import datetime
import datetime as dt
from pandas import DataFrame
from math import sqrt
import warnings
warnings.filterwarnings('ignore')

# Loading Datasets

In [ ]:
#ref: https://www.kaggle.com/fabiendaniel/elo-world
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Case Study 1/Data/Dataset/'

In [ ]:
# loading the datasets
%%time
card_details_train = pd.read_csv(dataset_path + 'card_details_train_clean.csv', parse_dates=['first_active_month'])
card_details_test = pd.read_csv(dataset_path + 'card_details_test_clean.csv', parse_dates=['first_active_month'])
#merchant_details = pd.read_csv(dataset_path + 'merchant_details_clean.csv')
old_transactions = pd.read_csv(dataset_path + 'old_transactions_clean.csv')
new_transactions = pd.read_csv(dataset_path + 'new_transactions_clean.csv')

print('All preprocessed dataset loaded successfully!')

All preprocessed dataset loaded successfully!
CPU times: user 57.5 s, sys: 12.2 s, total: 1min 9s
Wall time: 1min 26s


In [ ]:
# reduce the memory of all datasets
old_transactions = reduce_mem_usage(old_transactions)
new_transactions = reduce_mem_usage(new_transactions)

Mem. usage decreased to 1166.08 Mb (62.5% reduction)
Mem. usage decreased to 74.88 Mb (64.3% reduction)


<a name='section7'><a/>
# 7. Advance Feature Engineering

<a name='section7.1'><a/>
## 7.1. Dataset 1: card_details_train

In [ ]:
card_details_train['first_active_month'] = pd.to_datetime(card_details_train['first_active_month'])
card_details_train['month'] = card_details_train['first_active_month'].dt.month
card_details_train['year'] = card_details_train['first_active_month'].dt.year
card_details_train['dayofweek'] = card_details_train['first_active_month'].dt.dayofweek
card_details_train['weekofyear'] = card_details_train['first_active_month'].dt.weekofyear
card_details_train['elapsed_time'] = (datetime.datetime.today() - card_details_train['first_active_month']).dt.days

In [ ]:
#card_details_train = pd.get_dummies(card_details_train, columns=['feature_1', 'feature_2', 'feature_3'])

<a name='section7.2'><a/>
## 7.2. Dataset 2: card_details_test

In [ ]:
card_details_test['first_active_month'] = pd.to_datetime(card_details_test['first_active_month'])
card_details_test['month'] = card_details_test['first_active_month'].dt.month
card_details_test['year'] = card_details_test['first_active_month'].dt.year
card_details_test['dayofweek'] = card_details_test['first_active_month'].dt.dayofweek
card_details_test['weekofyear'] = card_details_test['first_active_month'].dt.weekofyear
card_details_test['elapsed_time'] = (datetime.datetime.today() - card_details_test['first_active_month']).dt.days

In [ ]:
#card_details_test = pd.get_dummies(card_details_test, columns=['feature_1', 'feature_2', 'feature_3'])

<a name='section7.3'><a/>
## 7.3. Dataset 4: old_transactions

In [ ]:
# ref: https://www.kaggle.com/chauhuynh/my-first-kernel-3-699/
%%time
old_transactions['purchase_date'] = pd.to_datetime(old_transactions['purchase_date'])
old_transactions['year'] = old_transactions['purchase_date'].dt.year
old_transactions['weekofyear'] = old_transactions['purchase_date'].dt.weekofyear
old_transactions['month'] = old_transactions['purchase_date'].dt.month
old_transactions['dayofweek'] = old_transactions['purchase_date'].dt.dayofweek
old_transactions['day'] = old_transactions['purchase_date'].dt.day
old_transactions['weekday'] = old_transactions.purchase_date.dt.weekday
old_transactions['weekend'] = (old_transactions.purchase_date.dt.weekday >=5).astype(int)
old_transactions['hour'] = old_transactions['purchase_date'].dt.hour
old_transactions['month_diff'] = ((datetime.datetime.today() - old_transactions['purchase_date']).dt.days)//30
old_transactions['month_diff'] += old_transactions['month_lag']

old_transactions['duration'] = old_transactions['purchase_amount']*old_transactions['month_diff']
old_transactions['amount_month_ratio'] = old_transactions['purchase_amount']/old_transactions['month_diff']
old_transactions['price'] = old_transactions['purchase_amount'] / old_transactions['installments']

CPU times: user 38 s, sys: 259 ms, total: 38.3 s
Wall time: 38 s


In [ ]:
# ref: https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
%%time
aggs = {}
aggs['purchase_amount'] = ['sum','max','min','mean','var','skew']
aggs['installments'] = ['sum','max','mean','var','skew']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','skew']
aggs['month_diff'] = ['max','min','mean','var','skew']
aggs['weekend'] = ['sum', 'mean']
aggs['weekday'] = ['sum', 'mean']
aggs['authorized_flag'] = ['sum', 'mean']
aggs['category_1'] = ['sum','mean', 'max','min']
aggs['card_id'] = ['size','count']
aggs['year'] = ['nunique']
aggs['month'] = ['nunique', 'mean', 'min', 'max']
aggs['hour'] = ['nunique', 'mean', 'min', 'max']
aggs['weekofyear'] = ['nunique', 'mean', 'min', 'max']
aggs['dayofweek'] = ['nunique']
aggs['day'] = ['nunique', 'mean', 'min', 'max']
aggs['subsector_id'] = ['nunique']
aggs['merchant_id'] = ['nunique']
aggs['merchant_category_id'] = ['nunique']
aggs['price'] = ['sum','mean','max','min','var']
aggs['duration'] = ['mean','min','max','var','skew']
aggs['amount_month_ratio'] = ['mean','min','max','var','skew']

for col in ['category_2','category_3']:
    old_transactions[col+'_mean'] = old_transactions.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean'] 

old_transactions_agg = old_transactions.groupby('card_id').agg(aggs)
old_transactions_agg.columns = ['old' + '_' + a + '_' + agg for a in aggs.keys() for agg in aggs[a]]
old_transactions_agg.reset_index(drop=False, inplace=True)

CPU times: user 7min 26s, sys: 11.3 s, total: 7min 38s
Wall time: 7min 21s


In [ ]:
# ref: https://www.kaggle.com/mfjwr1/simple-lightgbm-without-blending
old_transactions_agg['old_purchase_date_diff'] = (old_transactions_agg['old_purchase_date_max'] - old_transactions_agg['old_purchase_date_min']).dt.days
old_transactions_agg['old_purchase_date_average'] = old_transactions_agg['old_purchase_date_diff']/old_transactions_agg['old_card_id_size']
old_transactions_agg['old_purchase_date_uptonow'] = (datetime.datetime.today() - old_transactions_agg['old_purchase_date_max']).dt.days

In [ ]:
# merging the above new features with train and test dataset
train = pd.merge(card_details_train, old_transactions_agg, on='card_id', how='left')
test = pd.merge(card_details_test, old_transactions_agg, on='card_id', how='left')

<a name='section7.4'><a/>
## 7.4. Dataset 5: new_transactions

In [ ]:
%%time
new_transactions['purchase_date'] = pd.to_datetime(new_transactions['purchase_date'])
new_transactions['year'] = new_transactions['purchase_date'].dt.year
new_transactions['weekofyear'] = new_transactions['purchase_date'].dt.weekofyear
new_transactions['month'] = new_transactions['purchase_date'].dt.month
new_transactions['dayofweek'] = new_transactions['purchase_date'].dt.dayofweek
new_transactions['day'] = new_transactions['purchase_date'].dt.day
new_transactions['weekday'] = new_transactions.purchase_date.dt.weekday
new_transactions['weekend'] = (new_transactions.purchase_date.dt.weekday >=5).astype(int)
new_transactions['hour'] = new_transactions['purchase_date'].dt.hour
new_transactions['month_diff'] = ((datetime.datetime.today() - new_transactions['purchase_date']).dt.days)//30
new_transactions['month_diff'] += new_transactions['month_lag']

new_transactions['duration'] = new_transactions['purchase_amount']*new_transactions['month_diff']
new_transactions['amount_month_ratio'] = new_transactions['purchase_amount']/new_transactions['month_diff']
new_transactions['price'] = new_transactions['purchase_amount'] / new_transactions['installments']

CPU times: user 2.64 s, sys: 33.9 ms, total: 2.67 s
Wall time: 2.67 s


In [ ]:
%%time
aggs = {}
aggs['purchase_amount'] = ['sum','max','min','mean','var','skew']
aggs['installments'] = ['sum','max','mean','var','skew']
aggs['purchase_date'] = ['max','min']
aggs['month_lag'] = ['max','min','mean','var','skew']
aggs['month_diff'] = ['max','min','mean','var','skew']
aggs['weekend'] = ['sum', 'mean']
aggs['weekday'] = ['sum', 'mean']
aggs['authorized_flag']= ['sum', 'mean']
aggs['category_1'] = ['sum','mean', 'max','min']
aggs['card_id'] = ['size','count']
aggs['year'] = ['nunique']
aggs['month'] = ['nunique', 'mean', 'min', 'max']
aggs['hour'] = ['nunique', 'mean', 'min', 'max']
aggs['weekofyear'] = ['nunique', 'mean', 'min', 'max']
aggs['dayofweek'] = ['nunique']
aggs['day'] = ['nunique', 'mean', 'min', 'max']
aggs['subsector_id'] = ['nunique']
aggs['merchant_id'] = ['nunique']
aggs['merchant_category_id'] = ['nunique']
aggs['price'] = ['sum','mean','max','min','var']
aggs['duration'] = ['mean','min','max','var','skew']
aggs['amount_month_ratio'] = ['mean','min','max','var','skew']

for col in ['category_2','category_3']:
    new_transactions[col+'_mean'] = new_transactions.groupby([col])['purchase_amount'].transform('mean')
    aggs[col+'_mean'] = ['mean'] 

new_transactions_agg = new_transactions.groupby('card_id').agg(aggs)
new_transactions_agg.columns = ['new' + '_' + a + '_' + agg for a in aggs.keys() for agg in aggs[a]]
new_transactions_agg.reset_index(drop=False, inplace=True)

CPU times: user 5min 3s, sys: 12.3 s, total: 5min 15s
Wall time: 4min 54s


In [ ]:
new_transactions_agg['new_purchase_date_diff'] = (new_transactions_agg['new_purchase_date_max'] - new_transactions_agg['new_purchase_date_min']).dt.days
new_transactions_agg['new_purchase_date_average'] = new_transactions_agg['new_purchase_date_diff']/new_transactions_agg['new_card_id_size']
new_transactions_agg['new_purchase_date_uptonow'] = (datetime.datetime.today() - new_transactions_agg['new_purchase_date_max']).dt.days

In [ ]:
# merging the above new features with train and test dataset
train = pd.merge(train, new_transactions_agg, on='card_id', how='left')
test = pd.merge(test, new_transactions_agg, on='card_id', how='left')

In [ ]:
# ref: https://www.kaggle.com/chauhuynh/my-first-kernel-3-699/
train['old_first_buy'] = (train['old_purchase_date_min'] - train['first_active_month']).dt.days
train['new_first_buy'] = (train['new_purchase_date_min'] - train['first_active_month']).dt.days
train['card_id_total'] = train['new_card_id_size'] + train['old_card_id_size']
train['purchase_amount_total'] = train['new_purchase_amount_sum'] + train['old_purchase_amount_sum']

test['old_first_buy'] = (test['old_purchase_date_min'] - test['first_active_month']).dt.days
test['new_first_buy'] = (test['new_purchase_date_min'] - test['first_active_month']).dt.days
test['card_id_total'] = test['new_card_id_size'] + test['old_card_id_size']
test['purchase_amount_total'] = test['new_purchase_amount_sum'] + test['old_purchase_amount_sum']

In [ ]:
# checking for missing values after merging this new features with existing train and test set
print('------------------------Missing value in train set------------------------')
print(train.columns[train.isnull().any()])
print('='*80)
print('------------------------Missing value in test set------------------------')
print(test.columns[test.isnull().any()])

------------------------Missing value in train set------------------------
Index(['old_purchase_amount_skew', 'old_installments_skew',
       'old_month_lag_skew', 'old_month_diff_skew', 'old_price_sum',
       'old_price_mean', 'old_price_var', 'old_duration_skew',
       'old_amount_month_ratio_skew', 'new_purchase_amount_sum',
       'new_purchase_amount_max', 'new_purchase_amount_min',
       'new_purchase_amount_mean', 'new_purchase_amount_var',
       'new_purchase_amount_skew', 'new_installments_sum',
       'new_installments_max', 'new_installments_mean', 'new_installments_var',
       'new_installments_skew', 'new_purchase_date_max',
       'new_purchase_date_min', 'new_month_lag_max', 'new_month_lag_min',
       'new_month_lag_mean', 'new_month_lag_var', 'new_month_lag_skew',
       'new_month_diff_max', 'new_month_diff_min', 'new_month_diff_mean',
       'new_month_diff_var', 'new_month_diff_skew', 'new_weekend_sum',
       'new_weekend_mean', 'new_weekday_sum', 'new_weekday

In [ ]:
train['old_purchase_date_max'].fillna(train['old_purchase_date_max'].mode()[0], inplace=True)
train['old_purchase_date_min'].fillna(train['old_purchase_date_min'].mode()[0], inplace=True)
train['new_purchase_date_max'].fillna(train['new_purchase_date_max'].mode()[0], inplace=True)
train['new_purchase_date_min'].fillna(train['new_purchase_date_min'].mode()[0], inplace=True)

test['old_purchase_date_max'].fillna(test['old_purchase_date_max'].mode()[0], inplace=True)
test['old_purchase_date_min'].fillna(test['old_purchase_date_min'].mode()[0], inplace=True)
test['new_purchase_date_max'].fillna(test['new_purchase_date_max'].mode()[0], inplace=True)
test['new_purchase_date_min'].fillna(test['new_purchase_date_min'].mode()[0], inplace=True)

In [ ]:
for f in ['old_purchase_date_max','old_purchase_date_min','new_purchase_date_max','new_purchase_date_min']:
  train[f] = train[f].astype(np.int64) * 1e-9

for f in ['old_purchase_date_max','old_purchase_date_min','new_purchase_date_max','new_purchase_date_min']:
  test[f] = test[f].astype(np.int64) * 1e-9

In [ ]:
train['outliers'] = 0
train.loc[train['target'] < -30, 'outliers'] = 1

In [ ]:
for f in ['feature_1','feature_2','feature_3']:
    order_label = train.groupby([f])['outliers'].mean()
    train[f] = train[f].map(order_label)
    test[f] = test[f].map(order_label)



---



Save the final dataset

In [ ]:
train.shape, test.shape

((201917, 168), (123623, 166))

In [ ]:
train.to_csv(dataset_path + 'train_FE2.csv', index=False)
test.to_csv(dataset_path + 'test_FE2.csv', index=False)



---



# References:

*   https://www.kaggle.com/c/elo-merchant-category-recommendation
*   https://www.kaggle.com/c/elo-merchant-category-recommendation/discussion
*   https://www.kaggle.com/c/elo-merchant-category-recommendation/code
*   https://stackoverflow.com/
